In [20]:
!pip install scikit-learn==1.3.0

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.3 MB 6.7 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.3 MB 9.5 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.3 MB 10.3 MB/s eta 0:00:01
   ------------------------------------- -- 8.7/9.3 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


  You can safely remove it manually.


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

cust_df = pd.read_csv("data\santander-cs/train.csv", encoding='latin-1')
print('dataset shape:', cust_df.shape)
cust_df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


산탄데르 은행 고객 만족도조사 결과
피처 - 익명화, 컬럼명 익명화, 수치, 0인 데이터가 대다수 - 희소성
타겟 - TARGET

# 데이터 탐색

In [2]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


100명중 4명이 불만족하는 데이터

In [4]:
cust_df['TARGET'].value_counts() #0 만족, 1 불만족
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()

print(f'불만족 비율은 : {unsatisfied_cnt/total_cnt:.2f}')

불만족 비율은 : 0.04


최소값, Q1, Q2, Q3 가 0인 피처가 많다.
최소값 -999999.000000 인 피처 var3 > Nan > 중위값/최빈값 2로 대체

In [5]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [6]:
cust_df.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
           ...  
 231           1
 188           1
 168           1
 135           1
 87            1
Name: var3, Length: 208, dtype: int64

## 피처 선택 - id 삭제, var3 은 최빈값 2로 대체

In [7]:
cust_df.drop('ID', axis=1, inplace=True)
cust_df['var3'].replace(-999999, 2, inplace=True)

cust_df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


## X, y 데이터 분할 


In [8]:
X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]
X_features.shape, y_labels.shape

((76020, 369), (76020,))

## 훈련/테스트 분할

In [9]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
#X_train.count()
print(f'학습 데이터 shape : {X_train.shape}, 테스트 데이터 shape : {X_test.shape}')

train_cnt = y_train.count()
test_cnt = y_test.count()
print(f'학습 데이터 값의 비율 : {y_train.value_counts()/train_cnt}')
print(f'테스트 데이터  값의 비율 : {y_test.value_counts()/test_cnt}')

학습 데이터 shape : (60816, 369), 테스트 데이터 shape : (15204, 369)
학습 데이터 값의 비율 : 0    0.960964
1    0.039036
Name: TARGET, dtype: float64
테스트 데이터  값의 비율 : 0    0.9583
1    0.0417
Name: TARGET, dtype: float64


# DT 예측기

# RF 예측기

# XGBoost 예측기 p271

## 훈련 데이터 > 훈련:검증 분할

In [13]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

## XGBoost model

In [14]:
%%time

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156)
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

[09:39:05] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.82179	validation_1-auc:0.80068
[1]	validation_0-auc:0.83092	validation_1-auc:0.80941
[2]	validation_0-auc:0.83207	validation_1-auc:0.80903
[3]	validation_0-auc:0.83288	validation_1-auc:0.80889
[4]	validation_0-auc:0.83414	validation_1-auc:0.80924
[5]	validation_0-auc:0.83524	validation_1-auc:0.80907
[6]	validation_0-auc:0.83568	validation_1-auc:0.81005
[7]	validation_0-auc:0.83741	validation_1-auc:0.81088
[8]	validation_0-auc:0.83896	validation_1-auc:0.81305
[9]	validation_0-auc:0.83949	validation_1-auc:0.81363
[10]	validation_0-auc:0.83908	validation_1-auc:0.81277
[11]	v

0.8397812474965844

### 하이퍼오피트 파라미터 최적화

In [17]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.base import BaseEstimator, ClassifierMixin
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from hyperopt  import hp

In [15]:
# HyperOpt를 이용해 베이지안 최적화 기반으로 튜닝하기

In [18]:
from hyperopt.pyll.base import scope

# 하이퍼파라미터 검색공간
search_space = { 'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)), 
                'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
                'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                'subsample':hp.uniform('subsample', 0.5, 1.0),
                'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
                }

### 목적함수 만들기

3 Fold 교차 검증 이용, 평균 ROC-AUC 값 변환, -1 곱해 최대 ROC-AUC값이 최소 반환값 만들기

In [40]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score

def objective_func_xgb(params):
    
    model = XGBClassifier(
        n_estimators=100, #params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        #subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss'
    )
    # 3개 k-fold 방식으로 평가된 roc_auc  지표를 담는 list
    roc_auc_list = []

    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

        #early stopping 30회, 추출된 학습 검증 데이터로 xgbm 학습 수행
        model.fit(X_tr,y_tr, early_stopping_rounds=30, eval_metric="auc",
             eval_set = [(X_tr,y_tr), (X_val,y_val)])

        # 1로 예측한 확률값 추출 후 roc auc 계산, 평균 roc auc 계산 위해 list로 결과값 담기
        score = roc_auc_score(y_val, model.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
    
    # roc-auc 평균값 변수로 만들어 최솟값용 -1 곱해 리턴
    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -1*score_mean , 'status': STATUS_OK}

In [44]:
from hyperopt import fmin,tpe,Trials

trials = Trials()

best_params = fmin( 
    fn=objective_func_xgb,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,rstate=np.random.default_rng(42))



[0]	validation_0-auc:0.82848	validation_1-auc:0.80266 
[1]	validation_0-auc:0.83432	validation_1-auc:0.80502 
[2]	validation_0-auc:0.83955	validation_1-auc:0.80724 
[3]	validation_0-auc:0.84437	validation_1-auc:0.81049 
[4]	validation_0-auc:0.84671	validation_1-auc:0.81024 
[5]	validation_0-auc:0.85225	validation_1-auc:0.81208 
[6]	validation_0-auc:0.85530	validation_1-auc:0.81466 
[7]	validation_0-auc:0.85599	validation_1-auc:0.81488 
[8]	validation_0-auc:0.85743	validation_1-auc:0.81539 
[9]	validation_0-auc:0.86052	validation_1-auc:0.81741 
[10]	validation_0-auc:0.86237	validation_1-auc:0.81667
[11]	validation_0-auc:0.86427	validation_1-auc:0.81702
[12]	validation_0-auc:0.86804	validation_1-auc:0.81991
[13]	validation_0-auc:0.86924	validation_1-auc:0.81991
[14]	validation_0-auc:0.87113	validation_1-auc:0.82154
[15]	validation_0-auc:0.87361	validation_1-auc:0.82136
[16]	validation_0-auc:0.87628	validation_1-auc:0.82161
[17]	validation_0-auc:0.87938	validation_1-auc:0.82213
[18]	valid

Traceback (most recent call last):
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LYJ\AppData\Local\Temp\ipykernel_33112\3551913889.py", line 5, in <module>
    best_params = fmin(
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\hyperopt\fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "c:\Users\LYJ\anaconda3\envs\xgboost_env\lib\site-packages\hyperopt\fmin.py", line 300, in run
    self.serial_eval

In [45]:
best_params

NameError: name 'best_params' is not defined

### 최적의 파라미터로 xgbm 재학습,roc-auc 측정

In [16]:
best_model = XGBClassifier(
        n_estimators=int(best_params['n_estimators']),
        max_depth=int(best_params['max_depth']),
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss'
        tree_method='gpu_hist'
    )

KeyError: 'n_estimator'

In [ ]:
evals = [(X_tr,y_tr), (X_val,y_val)]
best_model.fit(X_tr,y_tr,
               early_stopping_rounds=50, eval_metric="auc",
               eval_set=evals, verbose=True)

xgb_roc_score2 = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
xgb_roc_score2

In [ ]:
### LGBM 모델 학습, 하이퍼 파라미터 튜닝

In [ ]:
# 최적화 파라미터 기준 모델 평가

n_estimator=500
learning_rate = 0.15
max_depth = 5


In [ ]:
from hyperopt.pyll.base import scope

# 하이퍼파라미터 검색공간
search_space = { 'n_estimators': hp.quniform('num_leaves', 32, 64, 1), 
                'max_depth': hp.quniform('max_depth', 100, 160, 1)),
                'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                'subsample': hp.uniform('subsample', 0.7, 1.0),
                'learning_rate': hp.uniform('colsample_bytree', 0.01, 0.2),
                }`  `

In [ ]:
#LGBM 추가
%%time
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimator=200, early_stopping_rounds=50)

evals = [(X_tr,y_tr), (X_val,y_val)]
lgbm_clf.fit(X_tr,y_tr,
               early_stopping_rounds=50, eval_metric="auc",


In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score, KFold
from lightgbm import LGBMClassifier

def objective_func_xgb(params):
    
    model = LGBMClassifier(
        n_estimators=100, #params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        #subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss'
    )
    #k-fold 추가
    roc_auc_list = []

    kf = KFold(n_split=3)
    for tr_index, val_index in kf.split(X_train)
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

        lgbm_clf.fit(X_tr,y_tr, early_stopping_rounds=30, eval_metric="auc",
        evals = [(X_tr,y_tr), (X_val,y_val)]

        score = roc_auc_score(y_val, lgbm_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)

    -1*np.mean(roc_auc_list)
    
    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -1*score_mean , 'status': STATUS_OK}

In [ ]:
1.auc 기준,평가 결과
2. 각각의 파라미터 값